In [1]:
# !pip install -U transformers datasets tqdm dotenv

In [2]:
import torch, torch.nn as nn
from transformers import AutoImageProcessor, AutoModel
from PIL import Image

In [3]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.mps.is_available() else "cpu"

model_repo = "facebook/dinov3-vits16-pretrain-lvd1689m"
processor = AutoImageProcessor.from_pretrained(model_repo)
backbone = AutoModel.from_pretrained(model_repo)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
from utils import load_fooddataset, test_accuracy

In [5]:
class DinoV3Linear(nn.Module):
    def __init__(self, backbone: AutoModel, hidden_size: int, num_classes: int, freeze_backbone: bool = True):
        super().__init__()
        self.backbone = backbone
        if freeze_backbone:
            for p in self.backbone.parameters():
                p.requires_grad = False
            self.backbone.eval()

        self.head = nn.Linear(hidden_size, num_classes)

    def forward(self, pixel_values):
        outputs = self.backbone(pixel_values=pixel_values)
        last_hidden = outputs.last_hidden_state
        cls = last_hidden[:, 0]
        logits = self.head(cls)
        return logits

    def count_params(self):
        total_params = sum(p.numel() for p in self.parameters())
        trainable_params = sum(p.numel() for p in self.parameters() if p.requires_grad)
        return total_params, trainable_params

    @property
    def device(self):
        return next(self.parameters()).device
        
# Setup Model
hidden_size = getattr(backbone.config, "hidden_size", None)
model = DinoV3Linear(backbone, hidden_size, num_classes=10, freeze_backbone=True).to(device) 
total_params, trainable_params = model.count_params()

# Setup Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.0004)

# Load Dataset
ds_train, ds_test = load_fooddataset(processor, batch_size_train = 128)

print(f"classifier model has {total_params/1e6:.2f}M parameters ({trainable_params} trainable)")

classifier model has 21.60M parameters (3850 trainable)


In [6]:
# Training Loop
step = 0

for epoch in range(10):
    for images, labels in ds_train:
        images, labels = images.to(device), torch.Tensor(labels).to(device).long()
        logits = model(images)
        loss = nn.functional.cross_entropy(logits, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    
        if step % 10 == 0:
            print(f"step {step} (epoch {epoch}), loss {loss.item():.2f}")
        if step % 100 == 0:
            model.eval()
            acc = test_accuracy(ds_test, model)
            print(f"step {step} (epoch {epoch}), accuracy {acc:.2f}%")
            model.train()
    
        step += 1
        # break

step 0 (epoch 0), loss 2.32


Test accuray: 100%|██████████| 21/21 [00:02<00:00,  9.64it/s]


step 0 (epoch 0), accuracy 15.45%
step 10 (epoch 0), loss 2.14
step 20 (epoch 0), loss 1.91
step 30 (epoch 0), loss 1.74
step 40 (epoch 0), loss 1.49
step 50 (epoch 0), loss 1.41
step 60 (epoch 0), loss 1.23
step 70 (epoch 0), loss 1.17
step 80 (epoch 0), loss 1.02
step 90 (epoch 0), loss 0.89
step 100 (epoch 1), loss 0.83


Test accuray: 100%|██████████| 21/21 [00:02<00:00,  9.67it/s]


step 100 (epoch 1), accuracy 95.45%
step 110 (epoch 1), loss 0.77
step 120 (epoch 1), loss 0.63
step 130 (epoch 1), loss 0.67
step 140 (epoch 1), loss 0.62
step 150 (epoch 1), loss 0.57
step 160 (epoch 1), loss 0.50
step 170 (epoch 1), loss 0.52
step 180 (epoch 1), loss 0.48
step 190 (epoch 2), loss 0.46
step 200 (epoch 2), loss 0.41


...

step 700 (epoch 7), accuracy 98.28%
step 710 (epoch 7), loss 0.13
step 720 (epoch 7), loss 0.10
step 730 (epoch 7), loss 0.09
step 740 (epoch 7), loss 0.15
step 750 (epoch 7), loss 0.10
step 760 (epoch 8), loss 0.12
step 770 (epoch 8), loss 0.09
step 780 (epoch 8), loss 0.15
step 790 (epoch 8), loss 0.15
step 800 (epoch 8), loss 0.07


Test accuray: 100%|██████████| 21/21 [00:02<00:00,  9.78it/s]


step 800 (epoch 8), accuracy 98.36%
step 810 (epoch 8), loss 0.10
step 820 (epoch 8), loss 0.11
step 830 (epoch 8), loss 0.11
step 840 (epoch 8), loss 0.09
step 850 (epoch 8), loss 0.07
step 860 (epoch 9), loss 0.08
step 870 (epoch 9), loss 0.10
step 880 (epoch 9), loss 0.13
step 890 (epoch 9), loss 0.10
step 900 (epoch 9), loss 0.10


Test accuray: 100%|██████████| 21/21 [00:02<00:00,  9.65it/s]


step 900 (epoch 9), accuracy 98.28%
step 910 (epoch 9), loss 0.08
step 920 (epoch 9), loss 0.08
step 930 (epoch 9), loss 0.06
step 940 (epoch 9), loss 0.09
